# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy, MedianStoppingPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, normal, choice
from azureml.train.automl import AutoMLConfig
from azureml.data.dataset_factory import TabularDatasetFactory

import os
import joblib
import pandas as pd
import sys

from sklearn.model_selection import train_test_split




In [2]:
import sys
!{sys.executable} -m pip install nltk

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()
experiment_name = 'capstone-hyperdrive'

experiment=Experiment(ws, experiment_name)

run = experiment.start_logging()

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [4]:
cpu_cluster_name = "notebook123222"
vm_size = "STANDARD_DS3_V2"

try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing cluster, use it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size, max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


In [5]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.5)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling( {

        "--C": uniform(0.1, 0.9),
        "--max_iter": choice(25, 50, 75, 100)
    })

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory="./", compute_target=compute_target, entry_script="train.py")

hyperdrive_run_config = HyperDriveConfig(hyperparameter_sampling=param_sampling, primary_metric_name="Accuracy", 
                                        estimator=estimator, policy=early_termination_policy,
                                        primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, max_total_runs=10)

In [6]:
#TODO: Submit your experiment
hd_run = experiment.submit(hyperdrive_run_config, show_output=True)

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
RunDetails(hd_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [8]:
best_run_hd = hd_run.get_best_run_by_primary_metric()
best_run_metrics = best_run_hd.get_metrics()
parameter_values = best_run_hd.get_details()

print("Best Run Id: ", best_run_hd.id)
print("Accuracy: ", best_run_metrics["Accuracy"])
print("Parameters: ", parameter_values["runDefinition"]["arguments"])

AttributeError: 'NoneType' object has no attribute 'get_metrics'

In [ ]:
#TODO: Save the best model

best_model_hd = best_run_hd.register_model(model_name="capstone-hd.pkl", model_path="./")

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
# compute_cluster.delete()